Detección de barras en galaxias - Modelos Alternativos

Proyecto integrador MNA

Integrantes

Jonathan Jesús Marmolejo Hernández - A01795195

Isaid Posadas Oropeza - A01795015

Luis Daniel Ortega Muñoz - A01795197


# Modelo de ensamble homogéneo Bagging con YOLOv8m
      

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Instalar YOLOv8

In [2]:
!pip install ultralytics > /dev/null 2>&1

In [3]:
from ultralytics import YOLO

# Verificar versión
import ultralytics
print(f"Ultralytics versión: {ultralytics.__version__}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics versión: 8.3.146


In [4]:
from ultralytics import settings
settings.reset()  # Reinicia configuraciones de cache si hay conflictos

# Ensamble Bagging con Voting

In [5]:
import os
import random
import shutil
from ultralytics import YOLO

# Configuración
dataset_base = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2'
output_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/bagging_sets'
num_sets = 4
seed = 42

# Eliminar carpeta anterior si existe
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

def clasificar_imagenes_por_clase(images_path, labels_path):
    con_barra = []
    sin_barra = []
    for img in os.listdir(images_path):
        if not img.endswith(('.jpg', '.png')):
            continue
        label_file = img.replace('.jpg', '.txt').replace('.png', '.txt')
        label_path = os.path.join(labels_path, label_file)
        if os.path.exists(label_path) and os.path.getsize(label_path) > 0:
            con_barra.append(img)
        else:
            sin_barra.append(img)
    return con_barra, sin_barra

def create_balanced_bootstrap_sets(dataset_path, output_dir, num_sets=4, seed=42):
    random.seed(seed)
    images_path = os.path.join(dataset_path, 'images/train')
    labels_path = os.path.join(dataset_path, 'labels/train')

    con_barra, sin_barra = clasificar_imagenes_por_clase(images_path, labels_path)
    min_class_len = min(len(con_barra), len(sin_barra))

    for i in range(num_sets):
        subset_dir = f'{output_dir}/bagging_set_{i}'
        images_subdir = f'{subset_dir}/images/train'
        labels_subdir = f'{subset_dir}/labels/train'
        if os.path.exists(subset_dir):
            shutil.rmtree(subset_dir)
        os.makedirs(images_subdir, exist_ok=True)
        os.makedirs(labels_subdir, exist_ok=True)

        sample_cb = random.choices(con_barra, k=min_class_len)
        sample_sb = random.choices(sin_barra, k=min_class_len)
        subset_images = sample_cb + sample_sb
        random.shuffle(subset_images)

        for img_name in subset_images:
            shutil.copy(os.path.join(images_path, img_name), os.path.join(images_subdir, img_name))
            label_name = img_name.replace('.jpg', '.txt').replace('.png', '.txt')
            shutil.copy(os.path.join(labels_path, label_name), os.path.join(labels_subdir, label_name))

# Generar subconjuntos balanceados
create_balanced_bootstrap_sets(dataset_base, output_dir, num_sets=num_sets, seed=seed)

# Entrenar modelos
for i in range(num_sets):
    data_yaml = f'{output_dir}/bagging_set_{i}/data.yaml'
    with open(data_yaml, 'w') as f:
        f.write(f'''
path: {output_dir}/bagging_set_{i}
train: images/train
val: {dataset_base}/images/val
test: {dataset_base}/images/test
names:
  0: bar
''')

    print(f'\n Entrenando modelo YOLOv8m en bagging_set_{i}...\n')
    model = YOLO('yolov8m.pt')
    model.train(
    data=data_yaml,
    imgsz=640,
    epochs=100,
    batch=32,
    device=0,  # A100
    name=f'bagging_yolov8m_{i}',
    conf=0.25,
    iou=0.45,
    patience=10,
    cache=True
)




 Entrenando modelo YOLOv8m en bagging_set_0...



100%|██████████| 49.7M/49.7M [00:00<00:00, 633MB/s]


Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/MNA_ProyectoIntegrador/bagging_sets/bagging_set_0/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.45, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=bagging_yolov8m_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overla

100%|██████████| 755k/755k [00:00<00:00, 66.6MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...

     29/100      15.8G     0.1191     0.7627     0.9264         51        640: 100%|██████████| 106/106 [00:22<00:00,  4.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:04<00:00,  4.27it/s]

                   all       1338        669      0.595      0.889      0.685      0.681
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 19, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



29 epochs completed in 0.233 hours.
Optimizer stripped from runs/detect/bagging_yolov8m_3/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/bagging_yolov8m_3/weights/best.pt, 52.0MB

Validating runs/detect/bagging_yolov8m_3/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:06<00:00,  3.49it/s]


                   all       1338        669      0.603      0.818      0.696       0.68
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/bagging_yolov8m_3


In [6]:
# Copiar todos los modelos entrenados a una carpeta
final_models_dir = '/content/modelos_bagging'
os.makedirs(final_models_dir, exist_ok=True)

for i in range(num_sets):
    src = f'runs/detect/bagging_yolov8m_{i}/weights/best.pt'
    dst = f'{final_models_dir}/bagging_yolov8m_{i}.pt'
    if os.path.exists(src):
        shutil.copy(src, dst)

print("Modelos guardados en:", final_models_dir)

Modelos guardados en: /content/modelos_bagging


## Ensamble Bagging con Voting

### Bagging con Voting con umbral de confianza de 0.35, IoU de 0.5 y número de votos mínimo de 3 de los 4 modelos

In [13]:
from google.colab import drive
drive.mount('/content/drive')

from ultralytics import YOLO
import os
import cv2
import numpy as np
from tqdm import tqdm
import shutil
import pandas as pd

# === Umbral de confianza mínimo ===
CONFIDENCE_THRESHOLD = 0.35

# === Rutas ===
modelos_bagging_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_bagging'
model_paths = [os.path.join(modelos_bagging_dir, f'bagging_yolov8m_{i}.pt') for i in range(4)]

image_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2/images/val'
output_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results'
csv_output_path = os.path.join(output_dir, 'predicciones_ensemble_bagging.csv')

# === Limpiar carpeta de salida ===
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# === Cargar modelos ===
models = [YOLO(p) for p in model_paths]

# === Funciones auxiliares ===
def iou(box1, box2):
    xA, yA = max(box1[0], box2[0]), max(box1[1], box2[1])
    xB, yB = min(box1[2], box2[2]), min(box1[3], box2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return interArea / float(box1Area + box2Area - interArea + 1e-6)

def ensemble_predictions(results_list, iou_thr=0.5, vote_threshold=3):
    all_boxes = []
    for model_idx, result in enumerate(results_list):
        for box in result.boxes:
            conf = box.conf[0].cpu().item()
            if conf < CONFIDENCE_THRESHOLD:
                continue
            coords = box.xyxy[0].cpu().numpy()
            cls = int(box.cls[0].cpu().item())
            all_boxes.append((coords, conf, cls, model_idx))

    grouped = []
    used = set()

    for i in range(len(all_boxes)):
        if i in used:
            continue
        group = [all_boxes[i]]
        model_ids = {all_boxes[i][3]}
        for j in range(i + 1, len(all_boxes)):
            if j in used:
                continue
            if all_boxes[i][2] == all_boxes[j][2] and iou(all_boxes[i][0], all_boxes[j][0]) >= iou_thr:
                if all_boxes[j][3] not in model_ids:
                    group.append(all_boxes[j])
                    model_ids.add(all_boxes[j][3])
                    used.add(j)
        used.add(i)

        if len(model_ids) >= vote_threshold:
            boxes_np = np.array([g[0] for g in group])
            avg_box = boxes_np.mean(axis=0)
            avg_conf = np.mean([g[1] for g in group])
            cls = group[0][2]
            grouped.append((avg_box, avg_conf, cls, len(model_ids)))

    final_detections = []
    for i, (box_i, conf_i, cls_i, votes_i) in enumerate(grouped):
        conflict = False
        for j, (box_j, conf_j, cls_j, votes_j) in enumerate(grouped):
            if i == j:
                continue
            if iou(box_i, box_j) >= iou_thr and cls_i != cls_j:
                if votes_i > votes_j or (votes_i == votes_j and conf_i > conf_j):
                    continue
                else:
                    conflict = True
                    break
        if not conflict:
            final_detections.append((box_i, conf_i, cls_i, votes_i))

    return final_detections

def convert_to_yolo_format(box, img_width, img_height):
    x1, y1, x2, y2 = box
    x_center = (x1 + x2) / 2.0 / img_width
    y_center = (y1 + y2) / 2.0 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

# === Procesar imágenes ===
image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))]
csv_data = []

for img_path in tqdm(image_paths):
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]
    results_list = [model(img)[0] for model in models]
    final_detections = ensemble_predictions(results_list)

    filename = os.path.splitext(os.path.basename(img_path))[0]
    cv2.imwrite(os.path.join(output_dir, f"{filename}.jpg"), img)

    with open(os.path.join(output_dir, f"{filename}.txt"), 'w') as f:
        for box, conf, cls, votes in final_detections:
            x_center, y_center, width, height = convert_to_yolo_format(box, img_width, img_height)
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {conf:.4f}\n")
            csv_data.append({
                "image": f"{filename}.jpg",
                "class": cls,
                "x_center": x_center,
                "y_center": y_center,
                "width": width,
                "height": height,
                "confidence": conf,
                "votes": votes
            })

# Guardar CSV
pd.DataFrame(csv_data).to_csv(csv_output_path, index=False)

print(f"\n Proceso completo. Revisa la carpeta:\n{output_dir}")
print(f"CSV resumen creado en:\n{csv_output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/1338 [00:00<?, ?it/s]


0: 640x640 1 bar, 10.7ms
Speed: 2.1ms preprocess, 10.7ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 2.0ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 bars, 9.6ms
Speed: 2.0ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.4ms
Speed: 2.0ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/1338 [00:01<41:24,  1.86s/it]


0: 640x640 1 bar, 10.0ms
Speed: 2.1ms preprocess, 10.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 1.9ms preprocess, 9.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 1.8ms preprocess, 9.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 1.9ms preprocess, 9.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.7ms
Speed: 2.1ms preprocess, 10.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.9ms
Speed: 1.8ms preprocess, 9.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.9ms
Speed: 1.7ms preprocess, 9.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 1.8ms preprocess, 9.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


...


0: 640x640 (no detections), 16.4ms
Speed: 2.9ms preprocess, 16.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.2ms
Speed: 2.1ms preprocess, 10.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.7ms
Speed: 2.0ms preprocess, 9.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.7ms
Speed: 2.7ms preprocess, 12.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.0ms
Speed: 1.8ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 bars, 10.6ms
Speed: 1.8ms preprocess, 10.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.9ms
Speed: 1.8ms preprocess, 9.9ms inference, 1.5ms postprocess per image at sh

100%|█████████▉| 1337/1338 [02:17<00:00, 10.20it/s]


0: 640x640 (no detections), 11.5ms
Speed: 1.7ms preprocess, 11.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.8ms
Speed: 1.8ms preprocess, 9.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.7ms
Speed: 1.8ms preprocess, 9.7ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 1.9ms preprocess, 9.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1338/1338 [02:17<00:00,  9.70it/s]


 Proceso completo. Revisa la carpeta:
/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results
CSV resumen creado en:
/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results/predicciones_ensemble_bagging.csv


In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score
import os

# Rutas actualizadas
gt_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2/labels/val'
pred_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results'

y_true = []
y_pred = []

# Archivos en el conjunto de validación
gt_files = [f for f in os.listdir(gt_dir) if f.endswith('.txt')]

for fname in gt_files:
    gt_path = os.path.join(gt_dir, fname)
    pred_path = os.path.join(pred_dir, fname)

    # === Ground truth ===
    with open(gt_path, 'r') as f:
        gt_lines = f.readlines()
    has_bar_gt = len(gt_lines) > 0
    y_true.append(1 if has_bar_gt else 0)

    # === Predicción del ensamble ===
    if os.path.exists(pred_path):
        with open(pred_path, 'r') as f:
            pred_lines = f.readlines()
        has_bar_pred = len(pred_lines) > 0
    else:
        has_bar_pred = False

    y_pred.append(1 if has_bar_pred else 0)

# === Métricas
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# === Resultados
print(" Evaluación del ensamble Voting (Bagging):")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


 Evaluación del ensamble Voting (Bagging):
Precision: 0.6463
Recall:    0.7892
F1-score:  0.7106


### Bagging con Voting con umbral de confianza de 0.35, IoU de 0.5 y número de votos mínimo de 2 de los 4 modelos

In [8]:
from google.colab import drive
drive.mount('/content/drive')

from ultralytics import YOLO
import os
import cv2
import numpy as np
from tqdm import tqdm
import shutil
import pandas as pd

# === Umbral de confianza mínimo ===
CONFIDENCE_THRESHOLD = 0.35

# === Rutas ===
modelos_bagging_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/modelos_bagging'
model_paths = [os.path.join(modelos_bagging_dir, f'bagging_yolov8m_{i}.pt') for i in range(4)]

image_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2/images/val'
output_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results'
csv_output_path = os.path.join(output_dir, 'predicciones_ensemble_bagging.csv')

# === Limpiar carpeta de salida ===
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# === Cargar modelos ===
models = [YOLO(p) for p in model_paths]

# === Funciones auxiliares ===
def iou(box1, box2):
    xA, yA = max(box1[0], box2[0]), max(box1[1], box2[1])
    xB, yB = min(box1[2], box2[2]), min(box1[3], box2[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return interArea / float(box1Area + box2Area - interArea + 1e-6)

def ensemble_predictions(results_list, iou_thr=0.5, vote_threshold=2):
    all_boxes = []
    for model_idx, result in enumerate(results_list):
        for box in result.boxes:
            conf = box.conf[0].cpu().item()
            if conf < CONFIDENCE_THRESHOLD:
                continue
            coords = box.xyxy[0].cpu().numpy()
            cls = int(box.cls[0].cpu().item())
            all_boxes.append((coords, conf, cls, model_idx))

    grouped = []
    used = set()

    for i in range(len(all_boxes)):
        if i in used:
            continue
        group = [all_boxes[i]]
        model_ids = {all_boxes[i][3]}
        for j in range(i + 1, len(all_boxes)):
            if j in used:
                continue
            if all_boxes[i][2] == all_boxes[j][2] and iou(all_boxes[i][0], all_boxes[j][0]) >= iou_thr:
                if all_boxes[j][3] not in model_ids:
                    group.append(all_boxes[j])
                    model_ids.add(all_boxes[j][3])
                    used.add(j)
        used.add(i)

        if len(model_ids) >= vote_threshold:
            boxes_np = np.array([g[0] for g in group])
            avg_box = boxes_np.mean(axis=0)
            avg_conf = np.mean([g[1] for g in group])
            cls = group[0][2]
            grouped.append((avg_box, avg_conf, cls, len(model_ids)))

    final_detections = []
    for i, (box_i, conf_i, cls_i, votes_i) in enumerate(grouped):
        conflict = False
        for j, (box_j, conf_j, cls_j, votes_j) in enumerate(grouped):
            if i == j:
                continue
            if iou(box_i, box_j) >= iou_thr and cls_i != cls_j:
                if votes_i > votes_j or (votes_i == votes_j and conf_i > conf_j):
                    continue
                else:
                    conflict = True
                    break
        if not conflict:
            final_detections.append((box_i, conf_i, cls_i, votes_i))

    return final_detections

def convert_to_yolo_format(box, img_width, img_height):
    x1, y1, x2, y2 = box
    x_center = (x1 + x2) / 2.0 / img_width
    y_center = (y1 + y2) / 2.0 / img_height
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    return x_center, y_center, width, height

# === Procesar imágenes ===
image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))]
csv_data = []

for img_path in tqdm(image_paths):
    img = cv2.imread(img_path)
    img_height, img_width = img.shape[:2]
    results_list = [model(img)[0] for model in models]
    final_detections = ensemble_predictions(results_list)

    filename = os.path.splitext(os.path.basename(img_path))[0]
    cv2.imwrite(os.path.join(output_dir, f"{filename}.jpg"), img)

    with open(os.path.join(output_dir, f"{filename}.txt"), 'w') as f:
        for box, conf, cls, votes in final_detections:
            x_center, y_center, width, height = convert_to_yolo_format(box, img_width, img_height)
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {conf:.4f}\n")
            csv_data.append({
                "image": f"{filename}.jpg",
                "class": cls,
                "x_center": x_center,
                "y_center": y_center,
                "width": width,
                "height": height,
                "confidence": conf,
                "votes": votes
            })

# Guardar CSV
pd.DataFrame(csv_data).to_csv(csv_output_path, index=False)

print(f"\n Proceso completo. Revisa la carpeta:\n{output_dir}")
print(f"CSV resumen creado en:\n{csv_output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/1338 [00:00<?, ?it/s]


0: 640x640 1 bar, 10.4ms
Speed: 2.2ms preprocess, 10.4ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.5ms
Speed: 2.1ms preprocess, 9.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 bars, 10.3ms
Speed: 2.1ms preprocess, 10.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.5ms
Speed: 2.0ms preprocess, 9.5ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/1338 [00:01<42:42,  1.92s/it]


0: 640x640 1 bar, 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.7ms
Speed: 1.8ms preprocess, 9.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.6ms
Speed: 1.7ms preprocess, 9.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.6ms
Speed: 1.9ms preprocess, 9.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.0ms
Speed: 1.9ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.8ms
Speed: 1.9ms preprocess, 9.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.6ms
Speed: 1.9ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.0ms
Speed: 1.7ms preprocess, 10.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


...


0: 640x640 (no detections), 10.5ms
Speed: 1.8ms preprocess, 10.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.9ms
Speed: 1.9ms preprocess, 9.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.0ms
Speed: 1.8ms preprocess, 10.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 11.3ms
Speed: 2.1ms preprocess, 11.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.5ms
Speed: 1.9ms preprocess, 10.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.5ms
Speed: 2.3ms preprocess, 10.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 bars, 9.5ms
Speed: 1.8ms preprocess, 9.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 9.8ms
Speed: 1.7ms preprocess, 9.8ms inference, 1.5ms postprocess per image at shap

100%|█████████▉| 1337/1338 [02:17<00:00, 10.32it/s]


0: 640x640 (no detections), 10.7ms
Speed: 2.1ms preprocess, 10.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 11.0ms
Speed: 1.8ms preprocess, 11.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.2ms
Speed: 1.8ms preprocess, 10.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 bar, 10.7ms
Speed: 1.8ms preprocess, 10.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 1338/1338 [02:18<00:00,  9.69it/s]


✅ Proceso completo. Revisa la carpeta:
/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results
📄 CSV resumen creado en:
/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results/predicciones_ensemble_bagging.csv


In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score
import os

# Rutas actualizadas
gt_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/galaxy_dataset_2/labels/val'
pred_dir = '/content/drive/MyDrive/MNA_ProyectoIntegrador/voting_bagging_results'

y_true = []
y_pred = []

# Archivos en el conjunto de validación
gt_files = [f for f in os.listdir(gt_dir) if f.endswith('.txt')]

for fname in gt_files:
    gt_path = os.path.join(gt_dir, fname)
    pred_path = os.path.join(pred_dir, fname)

    # === Ground truth ===
    with open(gt_path, 'r') as f:
        gt_lines = f.readlines()
    has_bar_gt = len(gt_lines) > 0
    y_true.append(1 if has_bar_gt else 0)

    # === Predicción del ensamble ===
    if os.path.exists(pred_path):
        with open(pred_path, 'r') as f:
            pred_lines = f.readlines()
        has_bar_pred = len(pred_lines) > 0
    else:
        has_bar_pred = False

    y_pred.append(1 if has_bar_pred else 0)

# === Métricas
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# === Resultados
print(" Evaluación del ensamble Voting (Bagging):")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


 Evaluación del ensamble Voting (Bagging):
Precision: 0.6024
Recall:    0.8879
F1-score:  0.7178
